In [1]:
!pip3 install -U selenium
!pip3 install webdriver-manager


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

In [2]:
def scrape(driver,url,outpath):

    #open a new csv writer
    fw=open(outpath,'w',encoding='utf8')
    writer=csv.writer(fw,lineterminator='\n')
    writer.writerow(['Date','Platform','Text','Person','Truthfulness'])
    
    driver.get(url) # visits the reviews url for the given product
    
    while True: # keep going until there are no more review pages
    
        


        reviews=driver.find_elements(by=By.CSS_SELECTOR, value='[class="o-listicle__item"]')


        for review in reviews: 
            
            Date, Platform, Text, Person, Truthfulness='NA','NA','NA','NA','NA'
#----------------------------------------------------------------------------------------------------------------------------  
#To find truthfulness
            img=review.find_elements(by=By.CSS_SELECTOR,value='img[class="c-image__thumb"]' )[1] 
            meter=img.get_attribute('alt')
            Truthfulness=meter
#----------------------------------------------------------------------------------------------------------------------------
            try: # try to find Date
                DateBox=review.find_element(by=By.CSS_SELECTOR, value='[class="m-statement__desc"]')
            except NoSuchElementException:
                DateBox=None
                
            #box found
            if DateBox:
                DateInfo=DateBox.get_attribute('innerHTML') # get the text of class attribute

                arr=DateInfo.split(" ")

                finalDate=""
                  
                if arr!=None:
                 finalDate=arr[2]+" "+arr[3]+arr[4]# extract rotten or fresh
                 if finalDate[len(finalDate)-1]==':':
                       finalDate=finalDate.rstrip(finalDate[-1]) 
                 Date=finalDate
                if arr!=None and len(arr)>5:
                    i=5
                    finalPlatform=""
                    while i < len(arr):
                       if arr[i]!='in' and arr[i]!="an" and arr[i]!="a":
                        finalPlatform+=arr[i]+" "
                       i+=1
                    Platform=finalPlatform
                        

#----------------------------------------------------------------------------------------------------------------------------- 
#-----------------------------------------------------------------------------------------------------------------------------
#For extracting Text

            try: # try to find the review article
                TextBox=review.find_element(by=By.CSS_SELECTOR, value='[class="m-statement__quote"]')
            except:
                TextBox=None
                
            #box found, extract text
            if TextBox: Text=TextBox.text
#----------------------------------------------------------------------------------------------------------------------------- 
#For extracting Name

            try: # try to find the review article
                PersonBox=review.find_element(by=By.CSS_SELECTOR, value='[class="m-statement__name"]')
            except:
                PersonBox=None
                
            #box found, extract text
            if PersonBox: Person=PersonBox.text
            

            
#------------------------------------------------------------------------------------------------------------------------------                
            writer.writerow([Date,Platform,Text,Person,Truthfulness])
#------------------------------------------------------------------------------------------------------------------------------



        button1=WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//a[text()="Next"]')))
        
        
        if '?page=202&category=truth-o-meter' in button1.get_attribute('href'): # final page reached, 'next' button is disabled on this page
            break
        driver.execute_script("arguments[0].click();", button1) 
        
      
        
        time.sleep(3)
    
    fw.close()
    

    

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [4]:
driver.maximize_window()

In [5]:

url='https://www.politifact.com/factchecks/list/?category=truth-o-meter'
outpath='data.csv'

In [6]:
scrape(driver,url,outpath)


KeyboardInterrupt

